In [19]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=0eaa1caf561f87dd826fbb6bc7a8075c8800a226dd5a006e725a04f2aab3080f
  Stored in directory: /tmp/pip-ephem-wheel-cache-djy6t4vo/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [20]:
!pip install -U gensim

In [21]:
import gdown
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
%matplotlib inline 
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit

#NLTK
import nltk
from nltk.corpus import stopwords
#nltk.download("english")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
pd.set_option('display.max_columns', None)

**Задаа 0: Загрузим и предобработаем данные**

In [23]:
!gdown --id 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown --id 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 208MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:07<00:00, 90.9MB/s]


**movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**ratings.csv:** Ratings of the Full Dataset.

In [24]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
movies_metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [26]:
# удалим ошибочные знаечния

movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [27]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [28]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

In [29]:
# оставим только колонки, которые мы собираемся использовать

movies_metadata = movies_metadata[['id', 'original_title', 'overview']]

In [30]:
# удалим дубли, потому что они есть и очень мешаются

movies_metadata = movies_metadata.drop_duplicates()

In [31]:
movies_metadata = movies_metadata.reset_index(drop=True)

**1 Получим случайную матрицу эмбедингов для каждого фильма**

In [32]:
# из исходных данных берем айдишник и добавляю 30 случайных колонок

matrix_emb = pd.concat([movies_metadata[['id']], 
                        pd.DataFrame(np.random.random((movies_metadata.shape[0], 30)))], 
                       axis=1)

In [33]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей

matrix_emb['id'] = matrix_emb['id'].astype(int)

In [34]:
matrix_emb.head(2)

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,862,0.277486,0.320194,0.943969,0.426013,0.412821,0.504561,0.487385,0.566324,0.942918,0.088905,0.190523,0.412219,0.737887,0.923994,0.731941,0.245043,0.221829,0.004308,0.223503,0.453099,0.532991,0.764871,0.003166,0.612602,0.417980,0.065777,0.305745,0.776013,0.502171,0.243559
1,8844,0.975965,0.607443,0.320093,0.843562,0.277178,0.004492,0.240246,0.711943,0.888897,0.204318,0.676739,0.848127,0.697727,0.436649,0.088274,0.133324,0.632106,0.962877,0.910658,0.014866,0.669063,0.642904,0.427894,0.324080,0.789951,0.737703,0.868006,0.882028,0.357214,0.381147


**2 Теперь напишем функцию train_test_split по юзеру по матрице рейтингов (юзер х фильмы = рейтинг)**

**Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях**


In [35]:
ratings = pd.read_csv('ratings.csv')

In [36]:
ratings.sample(5)

,userId,movieId,rating,timestamp
8420035,86736,32587,0.5,1446999013
6608212,68192,8604,4.0,1236281422
18720434,194452,65685,2.5,1430844522
17655991,183118,7484,3.0,1093054049
4149852,43105,2075,3.0,995889353


In [37]:
# оказвается, только к 7565 фильмам из movies_metadata есть оценки в ratings

common_ids = set(matrix_emb.id).intersection(set(ratings.movieId))

len(common_ids)

7565

In [38]:
# выберем только пересекающиеся фильмы

matrix_emb = matrix_emb[matrix_emb.id.isin(common_ids)]

In [39]:
matrix_emb.shape

(7565, 31)

In [40]:
ratings = ratings[ratings.movieId.isin(common_ids)]

In [41]:
len(ratings.movieId.unique())

7565

In [42]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [43]:
# вторым параметром на вход можно подать любой список пользователей

splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:800], 0.7)

**3 Для каждого юзера по train-фильмам находим средний вектор эмбединг через mean**

**Надо дополнительно домножить эмбединг-вектор на оценку фильма**

In [44]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['id'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['id']) \
                                  .reset_index() \
                                  .rename(columns={'index':'rating'}), on='id')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['id'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .mean())
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [45]:
# получим матрицу пользователей

res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [46]:
# преобразуем к табличному виду

res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'id'})

In [47]:
res_vec_df.head(2)

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,1,2.129097,2.398629,1.890708,1.718541,1.954059,1.803712,2.035492,2.351768,2.228141,1.300310,1.804043,2.714607,2.232916,3.106972,2.562163,2.403795,1.190928,2.097130,1.656913,1.941453,1.420550,2.811705,1.931086,1.652143,1.440384,2.364429,1.748901,1.900741,1.952044,1.472949
1,2,2.069808,1.686250,1.823098,1.912312,1.687370,2.204617,1.723503,1.471365,1.959478,1.256324,1.454156,1.712305,1.671176,1.670026,2.012372,1.815674,1.643054,2.421034,2.165428,2.476973,1.832857,1.394168,1.867956,1.933735,1.685100,1.675677,2.046856,2.299188,1.334459,1.884075


**Задача 4: Найдем топ близких фильмов, которые пользователь еще не смотрел (косинусное расстояние)**

In [48]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, splitted_data_dict, top_n):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['id'] == user_id] \
                                .drop('id', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['id']:
    
    film_vec = films_emb_mtrx[films_emb_mtrx['id'] == film].drop('id', axis=1).values.flatten().tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in splitted_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  return heapq.nlargest(top_n, scores, key=scores.get)

In [64]:
top_films = get_closest_films(res_vec_df, 11, matrix_emb, splitted_data, 50)

**Задача 5: Проверить результаты по метрикам**

In [ ]:
for user in splitted_data.keys():
  top_films = get_closest_films(res_vec_df, user, matrix_emb, splitted_data, 50)
  print(user, set(top_films).intersection(set(splitted_data[user][2])))

1 set()
2 set()
3 set()
4 set()
5 set()
6 set()
7 set()
8 set()
9 set()
10 set()
11 set()
12 set()
13 set()
14 set()
15 set()
16 set()
17 set()
18 set()
19 set()
20 set()
21 set()
22 set()
23 set()
24 set()
25 set()
26 set()
27 set()
28 set()
29 set()
30 set()
31 set()
32 set()
33 set()
34 set()
35 set()
36 set()
37 set()
38 set()
39 set()
40 set()
41 set()
42 set()
43 {2124}
44 set()
46 set()
47 set()
48 set()
49 {172}
50 set()
51 set()
52 set()
53 set()
54 {4154}
55 set()
56 set()
57 set()
58 set()
59 set()
60 {7454}
61 set()
62 set()
63 set()
64 set()
65 set()
66 set()
67 set()
68 {172}
69 set()
70 set()
71 set()
72 set()
73 set()
74 set()
75 set()
76 set()
77 set()
78 set()
79 set()
80 set()
81 set()
82 set()
83 set()
84 set()
85 set()
86 set()
87 set()
88 set()
89 set()
90 set()
91 set()
92 set()
93 set()
94 set()
95 set()
96 set()
97 set()
98 set()
99 set()
100 set()
101 set()
102 set()
103 set()
104 set()
105 set()
106 set()
107 {2861}
108 set()
109 set()
110 set()
111 set()
112

In [63]:
set(top_films).intersection(set(splitted_data[231][2]))

set()

In [ ]:
top_films

[65642,
 102419,
 27771,
 1422,
 26567,
 92307,
 172,
 27624,
 2124,
 1280,
 86412,
 2756,
 26587,
 5482,
 99367]

In [ ]:
splitted_data[11][2]

[53129,
 750,
 457,
 53519,
 527,
 59362,
 2959,
 46948,
 54503,
 1127,
 1573,
 51935,
 52458,
 318,
 3513,
 36513,
 51540,
 44555,
 500,
 4226,
 47518]

In [50]:
def precision_k(predictions_data, train_test_data, user_id, k):

  '''
     на вход подается:
     - predictions_data - матрица с train-предпочтениями пользователя
     - train_test_data - словарь с train_test data для всех пользователей
     - user_id - айди пользователя, для которого мы хотим получить оценку
     - k - количество рекомендуемых фильмов для оценки метрикой 

     на выходе:
     - знчаение метрики precision@k
  '''

  # находим количество пересекающихся элементов тестовых данных и предсказаний
  # и делим его на общее количество предсказываемых элементов
  TP = len(set(predictions_data).intersection(set(train_test_data[user_id][2])))

  return TP / k

In [ ]:
def recall_k():

**Часть Дианы:**

In [65]:
movies_df = pd.read_csv('movies_metadata.csv')[['id', 'original_title', 'overview']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
movies_df.head(2)

,id,original_title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...


In [7]:
movies_df.isnull().sum()

id                  0
original_title      0
overview          954
dtype: int64

In [8]:
movies_df = movies_df.fillna('')

In [9]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    """
       На вход: датафрейм с описанием и названием фильмов,
       также пробрасываются стоп-слова и класс для лемматизации текстов
      
       На выходе: список с предобработанными текстовыми данными
    """
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]
        # Lowercase & Drop stopwords and punctuation
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        # stem_text = " ".join([stemmer.stem(word) for word in stop_free.split(' ')])
        
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [10]:
nltk.download("stopwords")

text_overview = preprocessing(movies_df["overview"])
text_title = preprocessing(movies_df["original_title"])

movies_df["overview"] = text_overview
movies_df["original_title"] = text_title 

# Закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_title + text_overview

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/45466 [00:00<?, ?it/s]

  0%|          | 0/45466 [00:00<?, ?it/s]

In [11]:
text_prepr = [i.split() for i in text_prepr]

In [12]:
len(text_prepr)

90932

Word2Vec

In [13]:
model = Word2Vec(text_prepr, 
                 window=10, 
                 min_count=2)

In [14]:
model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 42232


Tockenizer & embeding

In [15]:
tokenizer = Tokenizer()

In [16]:
# Обновляем внутренний словарь на основе списка текстов.
tokenizer.fit_on_texts(text_prepr)
vocab_size = len(tokenizer.word_index) + 1

print("Total words:", vocab_size)

Total words: 77766


In [17]:
def embedding_matrix(tock, model):
    
    """
    На вход: словарь tokenizer.word_index 
    модель word2vec
    
    На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for word, i in tock.items():
        if word in model:
            embedding_matrix[i] = model[word]
    return embedding_matrix

In [55]:
id_film = movies_df['id'].nunique()

In [56]:
embedding_matrix = embedding_matrix(tokenizer.word_index, model.wv)

In [57]:
df_emb = pd.concat([movies_df[['id']],
                    pd.DataFrame(embedding_matrix)], axis=1)

In [59]:
df_emb.head(5)

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8844,-1.207218,1.895109,1.854302,0.603923,0.986595,-1.023746,1.450274,-0.669079,-0.437623,-0.344105,0.247835,-0.326354,0.491035,0.570331,0.117253,-0.593115,-1.686466,0.280476,-0.382791,-1.063853,0.339941,0.063649,-0.034821,-1.163491,-1.792694,0.895399,-0.528933,-0.157596,0.390677,-0.495474,-0.598703,1.133360,0.029166,-0.187988,-0.262782,1.105088,0.485917,-0.135757,0.241023,-0.961838,0.193122,-0.423300,-0.589728,-0.106035,0.518703,0.374947,-0.784715,-0.511394,1.218599,-0.027832,0.761112,-0.147480,-0.101967,-0.654471,-0.176418,0.661898,0.686996,0.543691,-0.203538,0.285953,0.052224,-0.283703,-1.144318,-0.580508,-0.112975,0.574956,0.482585,-0.493300,-0.248239,0.446420,-1.265307,0.220160,0.789494,-0.164119,1.053220,-0.340198,0.937136,0.282741,-0.692672,0.442647,-0.923523,2.053493,-0.444184,-0.151619,-0.057286,1.345384,-0.286877,1.097343,1.046199,-0.344983,0.490269,-0.854666,-0.454911,-0.681460,1.082227,-0.411277,-1.734929,-0.541368,0.826819,1.253782
2,15602,-1.325820,1.320564,0.605664,1.155887,0.438710,-0.816655,0.894587,0.460635,-0.589660,0.196800,-0.367477,-0.173598,0.445232,0.036026,0.300110,-0.384029,0.087919,-0.804220,-0.553364,-0.817470,-0.268971,0.528795,-0.040841,-0.461518,-0.777736,0.240685,-0.159846,-0.480053,-0.416486,0.075811,0.640128,0.355669,0.035033,0.603793,-0.400921,1.072641,-0.011279,0.475795,0.346721,-1.098713,0.283018,0.244329,-0.289102,-0.286712,0.044968,0.186601,-0.158146,0.618387,-0.166680,0.352727,-0.417634,-0.418673,-0.141059,-0.367363,0.863447,0.149119,0.402942,0.415116,0.034749,-0.133727,-0.739175,0.117484,-0.104738,-0.188862,-0.452401,0.415723,-0.067385,-0.179432,0.076808,0.533054,-0.625522,0.220913,0.381433,-0.343481,0.214687,-0.584361,0.495955,-0.133121,-0.046417,-0.224307,-0.010840,0.590043,0.134124,0.186878,-0.230956,0.784711,-0.703721,0.151060,1.043194,-0.423066,-0.446033,-0.174683,0.449638,0.068363,0.772275,0.101792,-0.734113,-0.409352,0.866234,0.335018
3,31357,-1.594422,1.390032,0.793172,0.971922,0.017790,-0.571108,0.414252,0.709278,-0.782738,0.485128,-0.627043,0.134234,-0.135938,-0.086044,0.468671,-0.156905,-0.064830,-1.059294,-0.179622,-0.938829,-0.243665,0.504169,-0.378342,-0.305101,-0.099457,-0.616824,0.493600,-0.198011,-0.356810,0.522598,1.514367,0.009117,-0.650452,0.707134,-0.144765,0.765788,-0.224422,0.907424,0.830471,-1.037299,0.362876,0.524056,0.101367,-0.168467,-0.346874,0.264715,-0.453490,1.063987,-0.619741,0.442337,-0.292340,-0.818331,-0.089972,-0.076009,1.025265,0.031848,0.191784,0.710061,-0.564697,0.017250,-0.716244,0.473819,-0.076526,-0.585202,-0.220565,0.326430,0.713565,-0.819979,0.965742,0.902250,-1.213344,0.039960,-0.005687,-0.364027,0.300477,-0.235553,0.112615,-0.381224,0.584617,-0.033537,0.864719,0.176983,0.230855,0.072459,-0.502430,0.972360,-1.609309,0.097865,0.985220,-0.324914,-0.270239,0.202890,0

In [60]:
df_emb.shape

(45466, 101)